## **Импорт библиотек**

In [380]:
import pandas as pd
import re
import matplotlib.pyplot as plt
!pip install pymorphy2
!pip install -U pymorphy2-dicts-ru
!git clone https://github.com/amueller/word_cloud.git
!pip install '/content/word_cloud'
import nltk
import pymorphy2
from nltk.corpus import stopwords
from collections import Counter
nltk.download('stopwords')
stopwords_ru = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

fatal: destination path 'word_cloud' already exists and is not an empty directory.
Processing ./word_cloud
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for wordcloud: filename=wordcloud-1.8.1.post9+g35ce9b7-cp37-cp37m-linux_x86_64.whl size=332926 sha256=7fd79e0c25d0398c4ee6c3839f2b27bc5858013396c311b930b313588c1b2d1e
  Stored in directory: /tmp/pip-ephem-wheel-cache-jbcn66t6/wheels/54/f5/77/89dfb02d5ae2d64f935572a2d1c518ef70729cb58cff47e659
Successfully built wordcloud
  Attempting uninstall: wordcloud
    Found existing installation: wordcloud 1.8.1.post9+g35ce9b7
    Uninstalling wordcloud-1.8.1.post9+g35ce9b7:
      Succes

## **Анализ данных**

In [381]:
data = pd.read_html("/content/Выгрузка 9 апреля.xls")
df = data[1]

Проверяем размер

In [382]:
df.shape

(4967, 8)

In [383]:
df.head()

,№ п/п,№документа,Датарегистрации,Исх. №Дата,Адресат,Автор,Краткоесодержание,Примечание
0,1,WSR/1-838/2019,09.04.2019,208-40908.04.2019,Уразов Р.Н.,Антонова С.В. (АО «Интерфакс»),Вх - О возможности использования сетевого изда...,NaN
1,2,WSR/1-837/2019,09.04.2019,05-33809.04.2019,Уразов Р.Н.,Картошкин С.А. (Министерство просвещения Росси...,Вх - Об участии в совещании по вопросу доработ...,NaN
2,3,WSR/1-835/2019,09.04.2019,И10-14/313709.04.2019,Уразов Р.Н.,Цивилев С.Е. (Администрация Кемеровской области),Вх - О командировании Саликовой К. и Глушко Д....,NaN
3,4,WSR/1-834/2019,09.04.2019,Д2/7121-ИС03.04.2019,Иванюк Л.А.,Волков Г.А. (Министерство транспорта Российско...,Вх - Об участии в Чемпионате мира по профессио...,NaN
4,5,WSR/1-833/2019,09.04.2019,15-20/06-91108.04.2019,Уразов Р.Н.,Атанов И.В. (Ставропольский государственный аг...,Вх - Об участии в церемонии открытия Вузовског...,NaN


Информация о датасете

In [384]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4967 entries, 0 to 4966
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   № п/п              4967 non-null   int64 
 1   №документа         4965 non-null   object
 2   Датарегистрации    4965 non-null   object
 3   Исх. №Дата         4967 non-null   object
 4   Адресат            4967 non-null   object
 5   Автор              4967 non-null   object
 6   Краткоесодержание  4967 non-null   object
 7   Примечание         308 non-null    object
dtypes: int64(1), object(7)
memory usage: 310.6+ KB


In [385]:
df.describe()

,№ п/п
count,4967.000000
mean,2484.000000
std,1433.993724
min,1.000000
25%,1242.500000
50%,2484.000000
75%,3725.500000
max,4967.000000


In [386]:
df[df['Датарегистрации'].str.contains("04.05.2018", na=False)]

,№ п/п,№документа,Датарегистрации,Исх. №Дата,Адресат,Автор,Краткоесодержание,Примечание
3215,3216,WSR/1.5-967/2018,04.05.2018,276/01-14/204.05.2018,Глушко Д.Е.,Гудков А.Н. (Департамент образования Ярославск...,Вх - О согласовании кандидатуры представителя ...,NaN
3216,3217,WSR/1.5-966/2018,04.05.2018,17-04/4644-ЕП04.05.2018,Глушко Д.Е.,Панькова Е.И. (Министерство образования и наук...,Вх - О подтверждении кандидатуры министра обра...,NaN
3217,3218,WSR/1-965/2018,04.05.2018,АГ-16/3470-И04.05.2018,Уразов Р.Н.,Никитин А.С. (Губернатор Новгородской области),Вх - О приглашении на Отборочные соревнования ...,NaN
3218,3219,WSR/1-964/2018,04.05.2018,2018-216/ПВ-Н04.05.2018,Уразов Р.Н.,Данилина Т.В. (Департамент образования города ...,Вх - О проведении плановой выездной проверки,NaN
3219,3220,WSR/1-963/2018,04.05.2018,377204.05.2018,Недоспасова О.В.,Адамова Н.В. (Министерство образования и науки...,Вх - Об участии в Чемпионате «LogisticSkills» ...,NaN
3220,3221,WSR/1.5-962/2018,04.05.2018,16-01-12/445104.05.2018,Глушко Д.Е.,Шевелева А.А. (Министерство образования Тульск...,Вх - О представителе в Региональном совете Союза,NaN
3221,3222,WSR/1.5-961/2018,04.05.2018,100-255004.05.2018,Глушко Д.Е.,Гимазутина Л.Н. (Министерство образования и на...,Вх - О внесении изменений в Приказ Союза №51 2...,NaN
3222,3223,WSR/1-960/2018,04.05.2018,ЦИК/1-56204.05.2018,Уразов Р.Н.,Грызлов Б.В. (Всероссийская политическая парти...,Вх - О приглашении к участию в заседании Высше...,NaN
3223,3224,WSR/1.5-959/2018,04.05.2018,39204.05.2018,Иванюк Л.А.,Маркачев Е.Е. (ГБПОУ Республики Мордовии Кремл...,Вх - Об отказе от участия в Отборочном чемпион...,NaN
3224,3225,WSR/1-958/2018,04.05.2018,ТС-1215/0603.05.2018,Уразов Р.Н.,Синюгина Т.Ю. (Министерство образования и наук...,Вх - О направлении плана работы на 2018 год по...,NaN


In [387]:
df[df['Автор'].str.contains("Администрация Президента", na=False)]

,№ п/п,№документа,Датарегистрации,Исх. №Дата,Адресат,Автор,Краткоесодержание,Примечание
511,512,WSR/1-326/2019,20.02.2019,АЧ-207313.02.2019,Уразов Р.Н.,Вайно А.Э. (Администрация Президента Российско...,Вх - Об участии в заседании в Гостином Дворе 2...,NaN
513,514,WSR/1-324/2019,20.02.2019,А26-01-НО-1070877113.02.2019,Гонтарь М.А.,Байло Е. (Администрация Президента Российской ...,Вх - Об участии в оглашении Президентом Россий...,NaN
553,554,WSR/1-283/2019,13.02.2019,924.01.2019,Уразов Р.Н.,Голикова Т.А. (Администрация Президента Россий...,Вх - О направлении плана работы и протокола ор...,NaN
626,627,WSR/1-211/2019,06.02.2019,А26-02-НО-561997129.01.2019,Гонтарь М.А.,Байло Е. (Администрация Президента Российской ...,Вх - О получении приглашения на Мировой Чемпио...,NaN
698,699,WSR/1-140/2019,28.01.2019,А8-821-225.01.2019,Уразов Р.Н.,Тямушкин И. (Администрация Президента Российск...,Вх - О предоставлении информации о выполнении ...,NaN
701,702,WSR/1-136/2019,28.01.2019,А26-02-НО-301077118.01.2019,Уразов Р.Н.,Байло Е. (Администрация Президента Российской ...,Вх - О проведении встречи Президента Российско...,NaN
704,705,WSR/1-133/2019,28.01.2019,ТГ-П8-41823.01.2019,Уразов Р.Н.,Голикова Т.А. (Администрация Президента Россий...,"Вх - Об организации пограничного, таможенного ...",NaN
1008,1009,WSR/1-3177/2018,12.12.2018,006412.12.2018,Уразов Р.Н.,Кириенко С.В. (Администрация Президента Россий...,Вх - Об посещении чемионата DigitalSkills в ка...,NaN
1027,1028,WSR/1-3157/2018,11.12.2018,А77-120607.12.2018,Никул Е.В.Кононова Н.А.,Липов А.Ю. (Администрация Президента Российско...,Вх - О проведении отраслевого чемпионата по ст...,NaN
1159,1160,WSR/1-3026/2018,27.11.2018,132/2307126.11.2018,Гонтарь М.А.,Кириенко С.В. (Администрация Президента Россий...,Вх - Об участии в межвузе 2018,NaN


In [388]:
df[-(df['Примечание'].isnull())]

,№ п/п,№документа,Датарегистрации,Исх. №Дата,Адресат,Автор,Краткоесодержание,Примечание
10,11,WSR/1.4-828/2019,08.04.2019,140-р08.04.2019,Крайчинская С.Б.,"Ажгиревич А.И. (ОООР ""Союз Машиностроителей Ро...",Вх - Об участии в заочном заседании Совета по ...,"Повестка заседания, а также материалы к нему, ..."
21,22,WSR/1-817/2019,08.04.2019,б/н05.04.2019,Уразов Р.Н.,Без автора (Управление Президента Российской Ф...,Вх - Об участии в заседании рабочей группы по ...,поступило с эл. адреса prm_Bilenkina@gov.ru в ...
25,26,WSR/1-813/2019,08.04.2019,ИП-398/0505.04.2019,Уразов Р.Н.,Потехина И.П. (Министерство просвещения Россий...,Вх - О предоставлении документов для заключени...,"срок до 6 апреля, письмо прислали 5 апр в 19:4..."
113,114,WSR/1-724/2019,01.04.2019,Д10/4932-ис07.03.2019,Уразов Р.Н.,Пашков К.А. (Министерство транспорта Российско...,Вх - Об участии в совещании по вопросу достиже...,письмо получено 01.04.2019 - (Почта России)
115,116,WSR/1-722/2019,01.04.2019,336525.03.2019,Глушко Д.Е.,"Томских А.А. (Министерство образования, науки ...",Вх - О направлении дорожной карты внедрения ре...,для передачи Уфимцеву/Дмитриеву
...,...,...,...,...,...,...,...,...
4935,4936,WSR/1.-1775/2017,03.11.2017,107/1027.10.2017,Уразов Р.Н.,"Климов Е.С. (ООО ""Эттон-Центр"")",Возражения на Отзыв Союза на Исковое завление,"Без приложений, т.к. письмо направлено в Арбит..."
4936,4937,WSR/1.-1774/2017,07.11.2017,2040/01/217.10.2017,Уразов Р.Н.,Равич К. (Министерство промышленной политики Р...,О приглашении на IV Чемпионат WorldSkills Hi-T...,Направлено Мироновой С.
4951,4952,WSR/1.-1761/2017,02.11.2017,03-15-1514/17-0-220.10.2017,Уразов Р.Н.,Соляников Ю.В. (Комитет по образованию Правите...,Об участии в мероприятиях IV Чемпионата WorldS...,Направлено Мироновой С.
4952,4953,WSR/1.-1758/2017,02.11.2017,4598/01-021-1701.11.2017,Уразов Р.Н.,Зубов Д.Ю. (Министерство образования и науки К...,Об участии в IV Национальном чемпионате сквозн...,Направлено Глушко Д.Е.


## **1.1 Предобработка данных и выделение значимых атрибутов**



In [389]:
df = data[1].drop(['№ п/п', '№документа', 'Исх. №Дата', 'Примечание'], axis=1)
df.head()

,Датарегистрации,Адресат,Автор,Краткоесодержание
0,09.04.2019,Уразов Р.Н.,Антонова С.В. (АО «Интерфакс»),Вх - О возможности использования сетевого изда...
1,09.04.2019,Уразов Р.Н.,Картошкин С.А. (Министерство просвещения Росси...,Вх - Об участии в совещании по вопросу доработ...
2,09.04.2019,Уразов Р.Н.,Цивилев С.Е. (Администрация Кемеровской области),Вх - О командировании Саликовой К. и Глушко Д....
3,09.04.2019,Иванюк Л.А.,Волков Г.А. (Министерство транспорта Российско...,Вх - Об участии в Чемпионате мира по профессио...
4,09.04.2019,Уразов Р.Н.,Атанов И.В. (Ставропольский государственный аг...,Вх - Об участии в церемонии открытия Вузовског...


## **1.2 Разбиение сложных атрибутов**

In [390]:
df = df.dropna(subset=['Датарегистрации'])

In [391]:
#Создаём новую колонку и добавляем в неё данные из старой
fio = r'([А-Я]\.[А-Я]\.\s?[А-Я][а-я]{1,20}|[А-Я][а-я]{1,20}\s?[А-Я]\.[А-Я]\.)'
list_adress = []
for i in df.Адресат:
  result = re.findall(fio, i)[-1]
  list_adress.append(result.replace(' ',' '))
df['Адресат(-ы)'] = list_adress
df.tail(2)
#Удаляем старую колонку
df = df.drop('Адресат',axis=1)

## **1.3 Дополнение недостаюими данными**

In [392]:
df['Агенство'] = df.Автор.apply(lambda x: x.split("(")[1].replace(')',''))

In [393]:
df['Автор'] = df.Автор.apply(lambda x: x.split("(")[0] + x.split(")")[1])

In [394]:
df.head()

,Датарегистрации,Автор,Краткоесодержание,Адресат(-ы),Агенство
0,09.04.2019,Антонова С.В.,Вх - О возможности использования сетевого изда...,Уразов Р.Н.,АО «Интерфакс»
1,09.04.2019,Картошкин С.А.,Вх - Об участии в совещании по вопросу доработ...,Уразов Р.Н.,Министерство просвещения Российской Федерации
2,09.04.2019,Цивилев С.Е.,Вх - О командировании Саликовой К. и Глушко Д....,Уразов Р.Н.,Администрация Кемеровской области
3,09.04.2019,Волков Г.А.,Вх - Об участии в Чемпионате мира по профессио...,Иванюк Л.А.,Министерство транспорта Российской Федерации
4,09.04.2019,Атанов И.В.,Вх - Об участии в церемонии открытия Вузовског...,Уразов Р.Н.,Ставропольский государственный аграрный универ...


In [395]:
list_agents = list(df['Агенство'])

## **1.5 Преобразование списков переадресаций**

In [396]:
agents = []
agents_number_one = ['Министерств', 'Правительств', 'Федеральн', 'Администр']
for i in df.Агенство:
  br = 0
  for t in agents_number_one:
    if br:
      break
    if t in i:
      br=1
      agents.append("Первые лица")
  if not(br):
    agents.append("Остальные")

## **1.4 Формирование словарей данных**

In [397]:
df['Тип_агенства'] = list(agents)
df.head()

,Датарегистрации,Автор,Краткоесодержание,Адресат(-ы),Агенство,Тип_агенства
0,09.04.2019,Антонова С.В.,Вх - О возможности использования сетевого изда...,Уразов Р.Н.,АО «Интерфакс»,Остальные
1,09.04.2019,Картошкин С.А.,Вх - Об участии в совещании по вопросу доработ...,Уразов Р.Н.,Министерство просвещения Российской Федерации,Первые лица
2,09.04.2019,Цивилев С.Е.,Вх - О командировании Саликовой К. и Глушко Д....,Уразов Р.Н.,Администрация Кемеровской области,Первые лица
3,09.04.2019,Волков Г.А.,Вх - Об участии в Чемпионате мира по профессио...,Иванюк Л.А.,Министерство транспорта Российской Федерации,Первые лица
4,09.04.2019,Атанов И.В.,Вх - Об участии в церемонии открытия Вузовског...,Уразов Р.Н.,Ставропольский государственный аграрный универ...,Остальные


In [398]:
type_agents = df.Тип_агенства.value_counts().reset_index().values
data = [i[1] for i in type_agents]
agents = ["Первым лицам", "Остальным"]
dict_agents = list(zip(agents, data))

### **Кол-во отправленных писем первым лицам и остальным агенствам**

In [399]:
dict_agents

[('Первым лицам', 2977), ('Остальным', 1988)]

In [400]:
print('Письма отправлялись {} раз {}, и {} раз {}'.format(dict_agents[0][1],dict_agents[0][0],dict_agents[1][1],dict_agents[1][0]))

Письма отправлялись 2977 раз Первым лицам, и 1988 раз Остальным


In [401]:
five = df.Агенство.value_counts()[:10]
dict_five_agents = {}
for i in five.reset_index().values:
  dict_five_agents[i[0]] = i[1]

### **Больше всего писем приходило данным рганизациям**

In [402]:
dict_five_agents

{'АНО "Дирекция спортивных и социальных проектов"': 42,
 'Автономная некоммерческая организация "Центр развития профессиональных компетенций"': 34,
 'Аппарат Правительства Российской Федерации': 60,
 'Комитет по образованию Правительства Санкт-Петербурга': 44,
 'Министерство образования Тульской области': 52,
 'Министерство образования и науки Республики Татарстан': 37,
 'Министерство образования и науки Российской Федерации': 82,
 'Министерство образования и науки Российской Федерации. Департамент государственной политики в сфере подготовки рабочих кадров и ДПО': 147,
 'Министерство общего и профессионального образования Свердловской области': 32,
 'Министерство просвещения Российской Федерации': 64}

In [403]:
five = df.Автор.value_counts()[:10]
dict_five = {}
for i in five.reset_index().values:
  dict_five[i[0]] = i[1]

### **Рейтинг авторов по отправленным письмам**

In [404]:
dict_five

{'Аникеев А.С. ': 25,
 'Губайдуллин Э.Ф. ': 49,
 'Ельцова Л.Ю. ': 37,
 'Картошкин С.А. ': 55,
 'Огородова Л.М. ': 31,
 'Потехина И.П. ': 66,
 'Соляников Ю.В. ': 45,
 'Черноскутова И.А. ': 112,
 'Шевелева А.А. ': 38,
 'Шохин А. ': 27}

### **Кол-во вхождений ключвых слов**

In [405]:
def normal_chars(word):
    wp = morph.parse(word)
    for i in wp:
        if 'NOUN' in i.tag and i.normal_form not in stopwords_ru:
            return i.normal_form
    return None

word_dict = Counter()

for i in df.Краткоесодержание.values:
  for a in i.split():
    normal_word = normal_chars(a)
    if normal_word is not None:
      word_dict[normal_word] += 1

In [406]:
top_ten = word_dict.most_common(10)
top_ten

[('участие', 1546),
 ('чемпионат', 1287),
 ('проведение', 1045),
 ('направление', 891),
 ('стандарт', 425),
 ('год', 416),
 ('компетенция', 415),
 ('программа', 409),
 ('проект', 389),
 ('заявка', 381)]

In [407]:
for i in top_ten:
  print("Слово '{0}' встречалось {1} раз".format(i[0], i[1]))

Слово 'участие' встречалось 1546 раз
Слово 'чемпионат' встречалось 1287 раз
Слово 'проведение' встречалось 1045 раз
Слово 'направление' встречалось 891 раз
Слово 'стандарт' встречалось 425 раз
Слово 'год' встречалось 416 раз
Слово 'компетенция' встречалось 415 раз
Слово 'программа' встречалось 409 раз
Слово 'проект' встречалось 389 раз
Слово 'заявка' встречалось 381 раз
